# Benchmarking the surface code

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

import stim
from pymatching import Matching

from hyperedge_decomposition import decompose_dem, from_dem_to_stim, from_stim_to_dem

In [10]:
rounds = 30
DISTANCE = 5
after_clifford_depolarization = 0.01
NUM_SHOTS = 1_000_000

In [11]:
print(f"round={rounds} \r", end="") 
circuit = stim.Circuit.generated(code_task="surface_code:rotated_memory_z", distance=DISTANCE, rounds=rounds, 
                                 after_clifford_depolarization=after_clifford_depolarization)

t0 = time.time()
dem = from_stim_to_dem(circuit.detector_error_model(decompose_errors=True))
decom_dem = decompose_dem(dem)
decom = from_dem_to_stim(decom_dem)
t1 = time.time()
print(f"round={rounds}   {(t1 - t0):0.3f}\r", end="") 

decom_stim = circuit.detector_error_model(decompose_errors=True)

sampler = circuit.compile_detector_sampler()
defects, log_flips = sampler.sample(shots=NUM_SHOTS, separate_observables=True)

MWPM = Matching(decom)
MWPM_stim = Matching(decom_stim)

t2 = time.time()
predictions = MWPM.decode_batch(defects)
predictions_stim = MWPM_stim.decode_batch(defects)
print(f"round={rounds}   {(t1 - t0):0.3f}   {(time.time() - t2):0.3f}") 

log_prob = np.average(predictions != log_flips)
log_prob_stim = np.average(predictions_stim != log_flips)

round=30   3.038   54.412


In [12]:
log_prob, log_prob_stim

(0.163391, 0.164938)